In [ ]:
import pandas as pd
import numpy as np
from scipy.optimize import linprog 
from numpy.linalg import solve
import urllib.request
import os

### Downloading and parsing data

In [ ]:
if not os.path.exists('data'):
    os.makedirs('data')

url = "http://www.nordpoolspot.com/globalassets/marketdata-excel-files/consumption-prognosis_2019_hourly.xls" 
urllib.request.urlretrieve(url, 'data/consumption.xls');

url2 = "http://www.nordpoolspot.com/globalassets/marketdata-excel-files/wind-power-dk-prognosis_2019_hourly.xls" 
urllib.request.urlretrieve(url2, 'data/wind-prognosis.xls');

consumptionProg_raw_data = pd.read_html('data/consumption.xls')[0].values
windProg_raw_data = pd.read_html('data/wind-prognosis.xls')[0].values

consumptionProg_df = pd.DataFrame(consumptionProg_raw_data)

consumptionProg_df.columns = ['Dates', 'Hours', 'NO', 'SE', 'FI', 'DK1', 'DK2', 'EE', 'LV', 'LT']
consumptionProg_df = consumptionProg_df[['Dates', 'Hours', 'DK1', 'DK2']]

windProg_df = pd.DataFrame(windProg_raw_data)
windProg_df.columns = ['Dates', 'Hours', 'DK1', 'DK2']

### Standard Model

In [ ]:
A_eq = np.array([[ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., 1.,  1., 
                  -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.]])
b_eq = 0

A = np.identity(A_eq.size)

power_g = np.array([120, 50, 200, 400, 60, 50, 60, 100, 70, 50, 70, 45, 50, 60, 50]) 
power_d = np.array([250, 300, 120, 80, 40, 70, 60, 45, 30, 35, 25, 10]) 
b = np.array([np.concatenate((power_g, power_d))]).T

cost_g = np.array([0, 0, 15, 30, 32.5, 34, 36, 37.5, 39, 40, 60, 70, 100, 150, 200])
cost_d = -np.array([200, 110, 100, 90, 85, 75, 65, 40, 38, 31, 24, 16])
c = np.concatenate((cost_g, cost_d))

res_std = linprog(c, A_eq=A_eq, b_eq = b_eq, A_ub=A, b_ub=b, bounds=(0, None))
print('Generation schedules:', res_std.x[:15])
print('Demand schedules:', res_std.x[15:])

### Dual

In [ ]:
a1 = np.ones(15)
a2 = -np.ones(12)
a3 = np.array([np.hstack((a1,a2))]).T
a4 = -np.identity(27)

A_tilde = np.concatenate((a4,a3),axis=1)

b_tilde = np.concatenate((cost_g, cost_d))

c_tilde = np.append(-b, np.array([0]))

res_dual = linprog(-c_tilde, A_ub=A_tilde, b_ub=b_tilde)
print('Cost:', res_dual.x[-1])